In [1]:
import dask.dataframe as dask_pd
import dask_geopandas as dask_gpd
import geopandas as gpd
import pandas as pd
from dask.distributed import Client
import time

In [2]:
dask_df_mdt =  dask_pd.read_csv('result/cr-result-xlarea.csv/*.part', assume_missing=True)

# dask_df_mdt = dask_df_mdt[(dask_df_mdt['absolute_rf_channel_number'] > 1200) & (dask_df_mdt['absolute_rf_channel_number'] < 1949)]

In [3]:
dask_df_mdt['isnumberic'] = dask_df_mdt['mobile_operator'].astype(str).str.isnumeric()

In [4]:
dask_df_mdt = dask_df_mdt[dask_df_mdt['isnumberic']==True]

In [ ]:
dask_df_mdt = dask_df_mdt[dask_df_mdt['mobile_operator']=='3']

In [5]:
dask_df_mdt.head(10)

,Unnamed: 0.1,Unnamed: 0,mobile_operator,created_date,latitude,longitude,cell_id,absolute_rf_channel_number,physical_cell_id,reference_signal_received_power,geometry,Provinsi,Kabupaten_Kota,isnumberic
61,2420.0,2420.0,3,2023-06-15 15:34:00.067,-6.255572,106.825753,2.147484e+09,NaN,200.0,NaN,POINT (106.8257532 -6.2555719),DKI JAKARTA,JAKARTA SELATAN,True
70,3538.0,3538.0,3,2023-06-16 18:08:00.476,-6.255505,106.825793,2.147484e+09,NaN,23.0,NaN,POINT (106.8257932 -6.2555054),DKI JAKARTA,JAKARTA SELATAN,True
71,3630.0,3630.0,3,2023-06-27 08:32:00.457,-6.255612,106.825763,2.147484e+09,NaN,200.0,NaN,POINT (106.825763 -6.2556122),DKI JAKARTA,JAKARTA SELATAN,True
80,6420.0,6420.0,3,2023-06-23 17:44:00.679,-6.255510,106.825715,5.178010e+07,NaN,363.0,NaN,POINT (106.8257154 -6.2555102),DKI JAKARTA,JAKARTA SELATAN,True
82,6687.0,6687.0,3,2023-06-23 10:24:00.974,-6.255503,106.825724,5.178012e+07,NaN,363.0,NaN,POINT (106.825724 -6.2555031),DKI JAKARTA,JAKARTA SELATAN,True
85,7431.0,7431.0,3,2023-06-16 18:08:00.476,-6.255505,106.825793,2.147484e+09,NaN,200.0,NaN,POINT (106.8257932 -6.2555054),DKI JAKARTA,JAKARTA SELATAN,True
93,9870.0,9870.0,3,2023-06-06 09:12:00.225,-6.255624,106.825720,2.147484e+09,NaN,23.0,NaN,POINT (106.8257197 -6.2556243),DKI JAKARTA,JAKARTA SELATAN,True
95,10066.0,10066.0,3,2023-06-16 18:08:00.466,-6.255505,106.825793,2.147484e+09,NaN,23.0,NaN,POINT (106.8257932 -6.2555054),DKI JAKARTA,JAKARTA SELATAN,True
479,4052.0,4052.0,3,2023-06-12 08:38:00.103,-6.188223,106.937521,5.171587e+07,NaN,397.0,NaN,POINT (106.9375205 -6.188223),DKI JAKARTA,JAKARTA TIMUR,True
483,4683.0,4683.0,3,2023-06-17 05:38:48.683,-6.170528,106.945435,5.210995e+07,NaN,93.0,NaN,POINT (106.9454347 -6.1705278),DKI JAKARTA,JAKARTA TIMUR,True


In [ ]:
dask_df_mdt['Band'] = 'Band-3'

In [ ]:
dask_df_mdt.head(100)

In [ ]:
dask_gdf_mdt = dask_gpd.from_dask_dataframe(dask_df_mdt, geometry=dask_df_mdt["geometry"].map_partitions(gpd.GeoSeries.from_wkt, meta=gpd.GeoSeries([])),)

dask_gdf_mdt = dask_gdf_mdt.set_crs(4326)

In [ ]:
badgrid = pd.read_csv('Grid_Folder/result-polygon-l1800-june23.csv')
badgrid = gpd.GeoDataFrame(badgrid, geometry=gpd.GeoSeries.from_wkt(badgrid['polygon_shape']), crs=4326)

In [ ]:
dask_gdf_mdt = dask_gdf_mdt.sjoin(badgrid, how='inner', predicate='within')